In [1]:
import nltk
import json
import random

## Carga de los datos

Punto de partida: un .json con una lista de proyectos asignados 

In [2]:
with open('../tmp/devex-corpus.json') as corpus_file:
    corpus = json.loads(corpus_file.read())

In [3]:
print(type(corpus))
print(corpus[0])

<type 'list'>
[u"GE-Tbilisi: Legislative impact assessment, drafting and representationService contract award noticeGeorgia1.Publication reference:EuropeAid/135035/DH/SER/GE.2.Publication date of the contract notice:30.4.2014.3.Lot number and lot title:1 lot only.4.Contract number and value:ENPI/2015/357121, 1 713 780 EUR.5.Date of award of the contract:23.1.2015.6.Number of tenders received:6.7.Overall score of chosen tender:99,54.8.Name, address and nationality of successful tenderer:IBF International Consulting SA, avenue Louise 209A, 1050 Brussels, BELGIUM.Center for International Legal Cooperation (CILC), Koninginnegracht 7, 2514 AA The Hague, NETHERLANDS.Deutsche Stiftung f\u0171r internationale rechtliche Zusammenarbeit e.V. (IRZ), Ubierstra\xdfe 92, 53173 Bonn, GERMANY.ISO code of country: BE.9.Duration of contract:30 months.10.Contracting authority:European Union, represented by the European Commission, on behalf of and for the account of the partner country, Tbilisi, GEORGIA.

## Preparado de los datos

En primer lugar, hay que limpiar el código HTML:

In [4]:
from bs4 import BeautifulSoup

corpus = [(BeautifulSoup(description, 'html.parser').get_text(), company) \
            for description, company in corpus ]

A continuación, vamos a convertir las descripciones en listas de palabras.

In [5]:
descriptions = []
for description, company in corpus:
    words = description.encode('utf-8').split()
    descriptions.append((words, company))

A partir de las descripciones, tenemos que extraer una lista de palabras características ("word features"). Será simplemente la lista total de palabras que aparecen en nuestras descripciones. Usamos las siguientes funciones:

In [6]:
def get_words_in_descriptions(descriptions):
    """
    Given a list of descriptions [([description words], "company name")], join all the
    words into a single list and return it.
    """
    all_words = []
    for (words, company) in descriptions:
      all_words.extend(words)
    return all_words

In [7]:
def get_word_features(wordlist):
    """Given a list of all the words, return a (dict_keys) list of unique words."""
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

(Una **FreqDist** es simplemente una cuenta de cuántas veces aparece cada palabra)

In [8]:
wordlist = nltk.FreqDist(get_words_in_descriptions(descriptions))
wordlist.most_common(10)

[('\xc2\xa0', 4599),
 ('of', 4480),
 ('the', 3538),
 ('and', 3525),
 (':', 3202),
 ('to', 1687),
 ('DURATION:', 1647),
 ('-', 1625),
 ('Contract', 1409),
 ('ACTION', 1398)]

In [9]:
word_features = get_word_features(get_words_in_descriptions(descriptions))
print(word_features)[100]

JEAN-JAQUES


## Creación del clasificador

Nuestro clasificador será una caja negra. Le proporcionaremos una descripción, y nos devolverá una clasificación (la empresa que más probable ganaría este contrato).

Nuestro clasificador necesitará un training set. Usaremos las primeras 500 descripciones como training set, y el resto como testing set.

In [10]:
# es importante tener grupos de training y testing homogéneos (mismas proporciones de cada empresa)
random.seed(54965)
random.shuffle(descriptions)

In [11]:
middle = len(descriptions)/2
training_set = descriptions[:middle]
testing_set = descriptions[middle:middle+middle/2]
print(len(descriptions))
print(len(training_set))
print(len(testing_set))

2500
1250
625


Para el clasificador no podemos usar nuestras descripciones directamente. Necesitamos una función (*feature extractor*) que sea capaz de extraer las features de la descripción. Esta función solo toma la descripción y devuelve nuestras *word features* indicando si son relevantes (**aproximación más básica: si aparecen o no en la descripción**).

In [12]:
def extract_features(description):
    """
    Feature extractor. Given a description (as a list of words), returns our list of word features,
    indicating whether each word is relevant for the description.
    """
    # remove duplicates
    description_words = set(description)
    features = {}
    # word_features is defined outside
    for word in word_features:
        features['contains({})'.format(word)] = (word in description_words)
    return features

In [13]:
extract_features('this is a project made in GUATEMALA'.split())

{'contains((SUFs))': False,
 'contains(waste)': False,
 'contains(Network)': False,
 'contains(ADMIN)': False,
 'contains(Parliamentary)': False,
 'contains(DALIAN)': False,
 'contains(lors)': False,
 'contains(distances)': False,
 'contains(Place)': False,
 'contains(TAVARA)': False,
 'contains(2010-08-11:)': False,
 'contains(FINALE)': False,
 'contains(fostering)': False,
 'contains(I.7)': False,
 'contains(Fischer,)': False,
 'contains(economically)': False,
 'contains(Donohue;)': False,
 'contains(2.3.2009.)': False,
 'contains(Portal)': False,
 'contains(Unfunded)': False,
 'contains(SIA)': False,
 "contains(JORDAN'S)": False,
 'contains(IRSN/GRS)': False,
 'contains(remained)': False,
 'contains(violence)': False,
 'contains(209A,)': False,
 'contains(Romania)': False,
 'contains(439.94)': False,
 'contains(Estado)': False,
 'contains(SENIOR)': False,
 'contains(COMBI)': False,
 'contains(Point,)': False,
 'contains(Aires,)': False,
 'contains(components),)': False,
 'contains(p

Usamos esta función para crear el training set, convirtiendo las descripciones en listas de features entendibles por el clasificador:

In [14]:
training_set = nltk.classify.apply_features(extract_features, training_set)
testing_set = nltk.classify.apply_features(extract_features, testing_set)

finalmente, creamos el clasificador, usando nuestro training set para entrenarlo

In [15]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [16]:
classifier.show_most_informative_features()

Most Informative Features
        contains(Amount) = True           Indivi : IBF In =     81.0 : 1.0
          contains(Date) = True           Indivi : IBF In =     74.6 : 1.0
       contains(support) = True           Chemon : Moore  =     69.3 : 1.0
        contains(Method) = True           Indivi : IBF In =     66.3 : 1.0
   contains(Description) = True           Indivi : IBF In =     65.7 : 1.0
       contains(Country) = True           Indivi : IBF In =     65.7 : 1.0
     contains(Reference) = True           Indivi : IBF In =     65.7 : 1.0
        contains(Number) = True           Indivi : IBF In =     65.1 : 1.0
            contains(EC) = True             UNDP : Ernst  =     64.2 : 1.0
          contains(Name) = True           Indivi : Moore  =     64.0 : 1.0


## Application of the classifier

Ahora podemos probar el clasificador. Intentemos clasificar una sola descripción:

In [17]:
"El primer contrato ha sido asignado a la empresa {}".format(testing_set[0][1])

'El primer contrato ha sido asignado a la empresa UNICEF'

Comprobemos si nuestro clasificador lo hace bien:

In [18]:
classifier.classify(testing_set[0][0])

u'UNICEF'

¡Uhm! Pero, ¿qué hay del resto? Creamos una función para comprobar los aciertos en **todo** el training set:

In [19]:
def check_accuracy(testing_set, classifier):
    """Given a training set and a classifier, apply it to all cases and return the percentage of hits."""
    hits = 0
    cases = 0
    for description, company in testing_set:
        if classifier.classify(description) == company:
            hits += 1
        cases += 1
    return (hits, cases, round(float(hits)/float(cases), 2))

In [20]:
hits, cases, accuracy = check_accuracy(testing_set[:100], classifier)
print("Nuestro clasificador acierta en el {}% de los casos".format(accuracy*100))

Nuestro clasificador acierta en el 44.0% de los casos
